In [257]:
import numpy as np

In [258]:
class NeuralNetwork:
    def __init__(self, layers: list):
        self.layers = layers
        for layer in self.layers:
            layer.init()

    def forward_pass(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward_pass(self, deriv, lr):
        for layer in reversed(self.layers):
            deriv = layer.backward(deriv, lr)


In [259]:
class Linear:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
    
    def init(self):
        self.W = np.random.uniform(-1, 1,(self.input_size,self.output_size))
        self.B = np.zeros((1, self.output_size))
    
    def forward(self, x):
        self.fw = x
        return np.dot(x, self.W) + self.B
    
    def backward(self, d, lr):
        d_w = np.dot(self.fw.T, d)
        d_e = np.dot(d, self.W.T)
        d_b = np.sum(d, axis=0, keepdims=True)
        self.W -= lr * d_w / self.fw.shape[0]
        self.B -= lr * d_b / self.fw.shape[0]
        return d_e

In [260]:
class ReLU:
    def __init__(self):
        pass

    def init(self):
        pass

    def forward(self, x):
        self.fw = x
        return x * (x > 0)
    
    def backward(self, d, lr):
        return d * np.where(self.fw > 0, 1.0, 0.0)

In [261]:
class Sigmoid:
    def __init__(self):
        pass
    
    def init(self):
        pass
    
    def forward(self, x):
        self.fw = x
        self.out = 1.0 / (1.0 + np.exp(-x))
        return self.out
    
    def backward(self, d, lr):
        return d * (self.out * (1.0 - self.out))

In [262]:
arch = [
    Linear(2,8),
    ReLU(),
    Linear(8,1),
    Sigmoid()
]
net = NeuralNetwork(arch)


In [263]:
def print_weights(self):
    for layer in self.layers:
        if type(layer) == Linear:
            print("Bias: ", layer.B)
            print("Weight: ", layer.W)

def mutate_weights(self):
    for layer in self.layers:
        if type(layer) == Linear:
            layer.W == np.random.normal(0,1, size=layer.W.size)
            layer.B == np.random.normal(0,1,size=layer.B.size)


NeuralNetwork.mutate_weights = mutate_weights
NeuralNetwork.print_weights = print_weights

In [264]:
net.print_weights()
net.mutate_weights()
net.print_weights()

Bias:  [[0. 0. 0. 0. 0. 0. 0. 0.]]
Weight:  [[-0.56665639  0.4799043   0.18144716  0.94932504 -0.45299388  0.8271851
  -0.2454456   0.70485382]
 [-0.53398728 -0.37805118 -0.42089356  0.6250212   0.63975699 -0.17919235
  -0.56074599 -0.70890409]]
Bias:  [[0.]]
Weight:  [[-0.28054597]
 [-0.15696125]
 [ 0.98110748]
 [ 0.00218825]
 [ 0.87014019]
 [ 0.23011515]
 [ 0.35283197]
 [ 0.71005892]]
Bias:  [[0. 0. 0. 0. 0. 0. 0. 0.]]
Weight:  [[-0.56665639  0.4799043   0.18144716  0.94932504 -0.45299388  0.8271851
  -0.2454456   0.70485382]
 [-0.53398728 -0.37805118 -0.42089356  0.6250212   0.63975699 -0.17919235
  -0.56074599 -0.70890409]]
Bias:  [[0.]]
Weight:  [[-0.28054597]
 [-0.15696125]
 [ 0.98110748]
 [ 0.00218825]
 [ 0.87014019]
 [ 0.23011515]
 [ 0.35283197]
 [ 0.71005892]]


/var/folders/fv/c34g947d7399z59488dvfyth0000gn/T/ipykernel_37122/1951010585.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  layer.W == np.random.normal(0,1, size=layer.W.size)


### Aufgabe 2

In [265]:
batch = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
labels = np.array([[0.0], [1.0], [1.0], [0.0]]) 

In [266]:
def fitness(net: NeuralNetwork):
    y_hat = net.forward_pass(batch)
    accuracy = np.mean(1 - np.abs(labels - y_hat))
    print(y_hat, accuracy)
    return accuracy

In [267]:
def elitist_selection(individuals: list[NeuralNetwork], n):
    f = np.zeros(shape=(len(individuals,)))
    for i, net in enumerate(individuals):
        f[i]= (fitness(net))
    return [individuals[i] for i in (f).argsort()[:n]]

In [268]:
popsize = 3
elite = 2
individuals = [NeuralNetwork([Linear(2,8), ReLU(), Linear(8,1), Sigmoid()]) for _ in range(popsize)]

In [269]:
x = elitist_selection(individuals, elite)  
for net in x:
    print(fitness(net))

[[0.5       ]
 [0.28473447]
 [0.62051682]
 [0.32623979]] 0.519752878319651
[[0.5       ]
 [0.42765289]
 [0.42932804]
 [0.35685615]] 0.5000311950041004
[[0.5       ]
 [0.5465229 ]
 [0.43959536]
 [0.44557407]] 0.5101360469701758
[[0.5       ]
 [0.42765289]
 [0.42932804]
 [0.35685615]] 0.5000311950041004
0.5000311950041004
[[0.5       ]
 [0.5465229 ]
 [0.43959536]
 [0.44557407]] 0.5101360469701758
0.5101360469701758
